# Post-processing of case studies

## Usage notes

All figures will be written to the directory `img` as pdf and png files.

I run this notebook on Ubuntu, using this version of matplotlib:
```
pip freeze | grep matplotlib
matplotlib==3.5.1
```
Some older versions return an error because they do not support some plot configurations.
I set in the virtual machine the RAM to 12GB as 8GB is not sufficient to parse two annual result files.

## Import required libraries

In [57]:
import os
import cases

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

import numpy as np


from buildingspy.io.outputfile import Reader
from buildingspy.io.postprocess import Plotter

Constants

In [2]:
AFlo = 111997 # Floor area in m2

## Configure plots

In [52]:
plt.rcParams['axes.facecolor']='whitesmoke'
plt.rcParams['font.size'] = 8
plt.rcParams['text.usetex'] = False
plt.rcParams['legend.facecolor'] = 'white'
plt.rcParams['legend.framealpha'] = 0.75
plt.rcParams['legend.edgecolor'] = 'none'
plt.rcParams['savefig.dpi'] = 300

def save_plot(figure, file_name):
    """ Save the figure to a pdf and png file in the directory `img`
    """
    import os
    import matplotlib.pyplot as plt

    out_dir = "img"
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    figure.savefig(os.path.join(out_dir, '{}.pdf'.format(file_name)))
    figure.savefig(os.path.join(out_dir, '{}.png'.format(file_name)))
    plt.clf()


def configure_axes(axes):
    """ Configure the axis style
    """
    axes.spines['right'].set_visible(False)
    axes.spines['top'].set_visible(False)
    axes.spines['left'].set_visible(False)
    axes.spines['bottom'].set_visible(False)
    axes.grid(color='lightgrey', linewidth=0.25)
    return

def get_results(case_name):
    """ Get the results for the case with name `case_name`
    """
    # Make sure simulation was successful
    dslog_name = os.path.join("simulations", case_name, "dslog.txt")
    with open(dslog_name) as dslog:
        if not "Integration terminated successfully" in dslog.read():
            raise Exception("Simulation failed. Check {}".format(dslog_name))
    file_name = cases.get_result_file_name(case_name)
    return Reader(file_name, "dymola")

def get_partial_results(case_name, list_of_variables):
    """ Get a dictionary with the variable names and the time series for `list_of_variables`
    """
    reader = get_results(case_name)
    d = dict()
    read_time = True
    for v in list_of_variables:
        if read_time:
            d['time'] = reader.values(v)[0]
            read_time = False
        d[v] = reader.values(v)[1]
    return d


# ---------------------------------------------------------------------------
# helper functions and scripts

def set_cases_and_initiate_plot():
    from matplotlib.gridspec import GridSpec
    cases = ['Base case', 'Guideline 36'] # fixme
    seasons = ['Winter', 'Spring', 'Summer']
    num_cases = len(cases)
    num_seasons = len(seasons)

    fig = plt.figure(figsize=(6.5,8.))
    gs1 = GridSpec(80, 1)
    gs1.update(left=0.1, right=0.9, hspace=0.05)

    ax = list()
    ax.insert(0, fig.add_subplot(gs1[0:11,:]))
    ax.insert(1, fig.add_subplot(gs1[12:23,:]))
    ax.insert(2, fig.add_subplot(gs1[28:39,:]))
    ax.insert(3, fig.add_subplot(gs1[40:51,:]))
    ax.insert(4, fig.add_subplot(gs1[56:67,:]))
    ax.insert(5, fig.add_subplot(gs1[68:79,:]))

    # fig, ax = plt.subplots(nrows=num_cases*num_seasons, ncols=1, figsize = (6.5,8.))
    # fig, ax = plt.subplots(nrows=20, ncols=1, figsize = (6.5,8.))

    return cases, seasons, num_cases, num_seasons, fig, ax

def set_title(ax, title):
    left, width = .01, .97
    bottom, height = .01, .88
    right = left + width
    top = bottom + height

    title_str = r"$\it{" + title + "}$"
    ax.text(left, top,
            title_str,
            verticalalignment = 'center',
            horizontalalignment = 'left',
            transform=ax.transAxes,
            fontsize = 6, color = 'k',
            bbox=dict(facecolor='white', alpha=0.75, edgecolor='none'))


def set_up_labels(i, ax, cases, seasons, num_cases, num_seasons, x_axis_label, y_axis_label):
    # Hide xtick labels and ticks on the upper case subplot (each basecase)
    if i % 2 == 0:
        hide_tick_labels(ax)

    # Print x axis title only below the lowest subplot
    if i  == num_cases*num_seasons - 1:
        ax.set_xlabel(x_axis_label)
    ax.set_ylabel(y_axis_label)
    #ax.xaxis.set_ticks(np.arange(min(t)+0, 365, 1))

    # Annotate case
    set_title(ax, cases[i % 2])
    # Annotate case
    # if i % 2 == 0:
    #     title_str = r"$\bf{" + seasons[i/2] + "}$" + ' (upper: ' + r"$\it{" + cases[i % 2] + "}$" + ', lower: ' + r"$\it{"  + cases[(i-1) % 2] + "}$" + ')'
    #     ax.set_title(title_str, # mg assign appropriate season/case
    #                  verticalalignment = 'top',
    #                  horizontalalignment = 'center',
    #                  fontsize = 6, color = 'k')

    # Print legend only at the lower plot (g36 case)
    if i % 1 == 0:
        ax.legend(loc='center right', ncol=1)
    configure_axes(ax)

    #plt.tight_layout(h_pad=0)
    plt.tight_layout()
    #plt.subplots_adjust(hspace = .2)

def tem_conv_CtoF(T_in_degC):
    '''Converts temperature provided in degC to degF
    '''
    T_in_degF = (T_in_degC)*9./5. + 32.

    return T_in_degF

def add_secondary_yaxis_for_degF(ax, time, temp_in_K):
        # Add a secondary axis with temperatures represented in F
        ax_F = ax.twinx()
        # Get limits to match with the left axis
        ax_F.set_ylim([tem_conv_CtoF(ax.get_ylim()[0]),tem_conv_CtoF(ax.get_ylim()[1])])
        # plot a "scaler" variable and make it invisible
        ax_F.plot(time, tem_conv_CtoF(temp_in_K-273.15), linewidth=0.0)
        ax_F.set_ylabel('temperature [$^\\circ$F]')
        configure_axes(ax_F)
        #ax.grid(False)
        #ax.xaxis.grid()

def hide_tick_labels(ax):
    '''Removes labels and ticks. Kwargs: bottom controls the ticks, labelbottom the tick labels
    '''
    ax.tick_params(axis = 'x',labelbottom='off',bottom='off')

## Read annual results

This section also clears the old results to free up memory.

In [42]:
# Free up storage, then read new data
import gc
#r_all_without_diverse_loads = list()
#r_all_with_diverse_loads = list()
#r_all = list()
gc.collect()

print("Cleaned up memory.")

Cleaned up memory.


Read result file. This takes around 3 minutes.

In [5]:
list_of_variables = ['ETot.y',
                     "CPUtime"]
#r_base=get_partial_results('base', list_of_variables)
r_base=get_results('base')
print("Read base results")


Read base results


In [6]:
print(f"Total energy use: {r_base.max('ETot.y')/3600/1E9:.2f} GWh")


Total energy use: 24.67 GWh


### Energy use


In [81]:
def plot_energy(results : list, case_name: list):
    from buildingspy.io.outputfile import Reader
    import matplotlib.pyplot as plt

    plt.clf()

    n = len(results)
    # Conversion from J to kWh/m2
    conv = 1/3600./1000./AFlo
    width = 0.5       # the width of the bars: can also be len(x) sequence

    EHeaPum = np.zeros(n)
    EComPla = np.zeros(n)
    EPumETS = np.zeros(n)
    EPumDis = np.zeros(n)
    EPumPla = np.zeros(n)
    EFanDry = np.zeros(n)
    EFanBui = np.zeros(n)
    EEleNon = np.zeros(n)
    EAllTot = np.zeros(n)


    idx = np.array([i for i in range(n)])
    for i in idx:
        res = results[i]

        EHeaPum[i]        = res.max('EHeaPum.y') * conv
        EComPla[i]        = res.max('EComPla.y') * conv
        EPumETS[i]        = res.max('EPumETS.y') * conv
        EPumDis[i]        = res.max('EPumDis.y') * conv
        EPumPla[i]        = res.max('EPumPla.y') * conv
        EFanDry[i]        = res.max('EFanDryCoo.y') * conv
        EFanBui[i]        = res.max('EFanBui.y') * conv
        EEleNon[i]        = res.max('EEleNonHvaETS.y') * conv
        EAllTot[i]        = res.max('ETot.y') * conv


    bottom = np.zeros(n)
    p0 = plt.bar(idx, EHeaPum, width, bottom=bottom)
    bottom = np.add(bottom, EHeaPum)
    p1 = plt.bar(idx, EComPla, width, bottom=bottom)
    bottom = np.add(bottom, EComPla)
    p2 = plt.bar(idx, EPumETS, width, bottom=bottom)
    bottom = np.add(bottom, EPumETS)
    p3 = plt.bar(idx, EPumDis, width, bottom=bottom)
    bottom = np.add(bottom, EPumDis)
    p4 = plt.bar(idx, EPumPla, width, bottom=bottom)
    bottom = np.add(bottom, EPumPla)
    p5 = plt.bar(idx, EFanDry, width, bottom=bottom)
    bottom = np.add(bottom, EFanDry)
    p6 = plt.bar(idx, EFanBui, width, bottom=bottom)
    bottom = np.add(bottom, EFanBui)
    p7 = plt.bar(idx, EEleNon, width, bottom=bottom)
    bottom = np.add(bottom, EEleNon)

    print(f"All electricity use = {EAllTot}")
    print(f"Sum of plot = {bottom}")
    np.testing.assert_allclose(EAllTot, bottom, err_msg="Expected energy to be the same.")

    plt.ylabel('site electricity use $\mathrm{[kWh/(m^2 \cdot a)]}$')
    plt.xticks(idx, ('base case', 'base case')) # fixme
    plt.tick_params(axis=u'x', which=u'both',length=0)

    #plt.yticks(np.arange(0, 81, 10))
    plt.legend((p0[0], p1[0], p2[0], p3[0], p4[0], p5[0], p6[0], p7[0]), \
               ('heat pump ETS', 'heat pump plant', 'pumps ETS', 'pumps district', 'pumps plant', 'fans plant', 'fans buildings', 'electricity buildings'), \
                bbox_to_anchor=(1.5, 1.0), loc='upper right')
    plt.tight_layout()

    save_plot(plt, "energy")

    # Write result to console and file
##    eSit = [0, 0]
##    for i in [0, 1]:
##        eSit[i] = hea[i]+cooSen[i]+cooLat[i]+fan[i]

##    str = """\
##.. table:: Heating, cooling, fan and total site HVAC energy, and savings of guideline 36 case versus base case.
##
##   ===================================== ===================================== ====================================== ====================================== =====
##   :math:`E_{{h}} \quad [kWh/(m^2\,a)]`    :math:`E_{{c}} \quad [kWh/(m^2\,a)]`    :math:`E_{{f}} \quad [kWh/(m^2\,a)]`     :math:`E_{{tot}} \quad [kWh/(m^2\,a)]`     [%]
##   ===================================== ===================================== ====================================== ====================================== =====
##   {:37.4} {:37.4}  {:37.4}  {:37.4}
##   {:37.4} {:37.4}  {:37.4}  {:37.4}  {:4.3}
##   ===================================== ===================================== ====================================== ====================================== =====
##
##  """.format(\
##            hea[0], coo[0], fan[0], eSit[0], \
##            hea[1], coo[1], fan[1], eSit[1], (1-eSit[1]/eSit[0])*100.)
    def save_rst(str, file_name):
        ''' Save the string `str` to the rst file `file_name.rst`
        '''
        print(str)
        with open(os.path.join("img", "{}.rst".format(file_name)), "w") as fil:
            fil.write(str)

##    save_rst(str, "site_energy")
plot_energy([r_base, r_base], ['base', 'base'])

All electricity use = [220.24158301 220.24158301]
Sum of plot = [220.24158674 220.24158674]


### Diagnostic output

In [8]:
print(f"CPUtime, base {r_base.max('CPUtime')/3600.:.2f} h")

CPUtime, base 16.64 h


## Loop temperatures

In [84]:
plt.clf()

(tP, TLooMin)     = r_base.values('cenPla.TLooMin')
(tP, TLooMax)     = r_base.values('cenPla.TLooMax')
(t, TLooMinMea)  = r_base.values('cenPla.TLooMinMea')
(t, TLooMaxMea)  = r_base.values('cenPla.TLooMaxMea')
(t, TSoiPer)     = r_base.values('dTSoiPer.T')
(t, TSoiCen)     = r_base.values('dTSoiCen.T')

fig = plt.figure()
ax = fig.add_subplot(211)
ax.plot(t/24./3600., TLooMinMea-273.15, 'b', label='Minimum loop temperature', linewidth=0.5)
ax.plot(t/24./3600., TLooMaxMea-273.15, 'r', label='Maximum loop temperature', linewidth=0.5)
ax.plot(t/24./3600., TSoiCen-273.15, 'k', label='Average temperature center borefield', linewidth=1.5)
ax.plot(t/24./3600., TSoiPer-273.15, 'g', label='Average temperature perimeter borefield', linewidth=1.5)

rect1 = matplotlib.patches.Rectangle((tP[0], 0), 365, TLooMin[0]-273.15, color='mistyrose')
ax.add_patch(rect1)
rect1 = matplotlib.patches.Rectangle((tP[0], TLooMax[0]-273.15), 365, 30, color='mistyrose')
ax.add_patch(rect1)

ax.set_xlabel('time [d]')
ax.set_ylabel(r'temperature [$^\circ$C]')
#ax.set_xticks(list(range(25)))
ax.set_xlim([0, 365])
ax.set_ylim([10, 25])
ax.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
ax.set_aspect(10)
ax.grid(True)

# Energy
(t, EETS)     = r_base.values('ETotEts.y')
(t, EHexDry)  = r_base.values('cenPla.EHexEne.y')
(t, EBorCen)  = r_base.values('dTSoiCen.E')
(t, EBorPer)  = r_base.values('dTSoiPer.E')
(t, EHPCen)   = r_base.values('cenPla.EHeaPumEne.y')
QPip = np.zeros(len(t))
for i in range(1, 6):
    (_, tmp)     = r_base.values(f'dis.heatPorts[{i}].Q_flow')
    QPip = np.add(QPip, tmp)

EPip = np.zeros(len(t))
for i in range(len(t)-1):
    EPip[i+1] = EPip[i] + (QPip[i+1]+QPip[i])/2.*(t[i+1]-t[i])

ax = fig.add_subplot(212)
ax.plot(t/24./3600., -EETS/3600./1E9,    'b', label='Energy from ETS heat exchanger', linewidth=0.5)
ax.plot(t/24./3600., EHexDry/3600./1E9, 'r', label='Energy from central plant economizer', linewidth=0.5)
ax.plot(t/24./3600., -EBorCen/3600./1E9, 'k-+', label='Energy from center borefield', linewidth=0.5, markevery=60000)
ax.plot(t/24./3600., -EBorPer/3600./1E9, 'k-*', label='Energy from perimeter borefield', linewidth=0.5, markevery=60000)
ax.plot(t/24./3600., EPip/3600./1E9,    'k-o', label='Energy from soil into distribution pipe', linewidth=0.5, markevery=60000)
ax.plot(t/24./3600., EHPCen/3600./1E9,  'g', label='Energy from central heat pump', linewidth=0.5)


ax.set_xlabel('time [d]')
ax.set_ylabel(r'energy [GWh]')
#ax.set_xticks(list(range(25)))
ax.set_xlim([0, 365])
ax.set_ylim([-11, 11])
ax.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
ax.grid(True)
plt.tight_layout()
#plt.title()

save_plot(plt, "loopTemperatures")


In [65]:
len(EBorPer)
range(1, 6)

range(1, 6)

In [15]:
344019/10

34401.9